In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sympy as sp
from printerdemo import MyPrinter, PrinterSetting, Variable, Pointer
from mockups import newton_raphson_algorithm as newton_algo
from mockups import newton_raphson_function as newton_func

In [ ]:
def my_ccode(expr, **kwargs):
    return MyPrinter(**kwargs).doprint(expr)

In [ ]:
x, dx, abstol = sp.symbols('x dx a')
expr = sp.cos(x) - x**3

In [ ]:
algorithm = newton_algo(expr, x, abstol, dx)

In [ ]:
print(my_ccode(algorithm))

In [ ]:
print(my_ccode(newton_algo(expr, x, debug=True, itermax=7)))

In [ ]:
prec = PrinterSetting('precision')
newton_c = my_ccode(newton_func(expr, x, atol=10**(1-prec), debug=True, itermax=10),
                    settings={'precision': 7})
print(newton_c)

In [ ]:
from pycompilation import compile_link_import_strings
mod = compile_link_import_strings([
    ('newton.c', '#include <math.h>\n#include <stdio.h>\n' + newton_c),
    ('_newton.pyx', """cdef extern {type} newton({type})
def py_newton(x):
    return newton(x)""".format(type=newton_c.split()[0]))], std='c99')

In [ ]:
mod.py_newton(.6), mod.py_newton(.5)

In [ ]:
mod.py_newton(.86) - 0.865474033102

In [ ]:
from mockups import assign_cse

In [ ]:
y = sp.Symbol('y')
pw1 = sp.Piecewise((x, sp.Lt(x, 1)), (x**2, True))
pw2 = sp.Piecewise((2/(x + 1), sp.Lt(x, 1)), (sp.exp(1-x), True))
scp = assign_cse(y, pw1 + pw2)
print(my_ccode(scp))

In [ ]:
sp.cse(pw1+pw2)

In [ ]:
k = sp.Symbol('k')
expr2 = sp.cos(k*x) - x**3
print(my_ccode(newton_func(expr2, x, (x, k), debug=True, itermax=7)))

In [ ]:
print(my_ccode(newton_func(expr2, x, (x, Variable(k, const=True)))))

In [ ]:
print(my_ccode(newton_func(expr2, x, (x, Pointer(k)))))